In [0]:
%sql
CREATE STREAMING LIVE TABLE bronze_orders
COMMENT "Raw orders data ingested incrementally from S3 using Auto Loader."
TBLPROPERTIES ("quality" = "bronze")
AS SELECT * FROM cloud_files(
  "s3://databricksmedallionsbronze/raw/", -- Replace with your actual S3 path
  "csv",
  map(
    "cloudFiles.schemaLocation", "/checkpoints/bronze_orders_schema/", -- Replace with your actual schema location
    "cloudFiles.format", "csv",
    "header", "true",
    "inferSchema", "true"
  )
);

In [0]:
df = (
  spark.readStream.format("cloudFiles")
  .option("cloudFiles.format", "csv")
  .option("header", "true")
  .option("cloudFiles.schemaLocation", "s3://databricksmedallionsbronze/schema/")
  .load("s3://databricksmedallionsbronze/raw/")
)

In [0]:
# Create the schema if it does not exisspark.sql("use general_catalog.bronze_schema")
spark.sql("use general_catalog.bronze_schema")
# Write the stream to the Delta table
df.writeStream.format("delta") \
  .option("checkpointLocation", "s3://databricksmedallionsbronze/schema/") \
  .trigger(availableNow=True) \
  .toTable("bronze_schema.orders")

In [0]:
%sql
select * from  bronze.orders